In [1]:
import pandas as pd
import numpy as np

In [2]:
dataframe = pd.read_excel('test.xlsx', sheet_name='Sheet1')
dataframe.head()

,MARK,CTN NO,DESCRIPTION,PCS/TOTAL,UNITS,PCS/CTN,CTN/TOTAL,CBM/TOTAL,WEIGHT/TOTAL,LOGO,BIS NO.,BIS MODEL NO.,MAH,MADE IN
0,PVC,1,BACK COVER,300.0,PCS,300.0,1.0,0.090,22.8,HELLO KITTY,NaN,NaN,NaN,NaN
1,PVC,2,BACK COVER,320.0,PCS,320.0,1.0,0.090,24.4,HELLO KITTY,NaN,NaN,NaN,NaN
2,PVC,3,BACK COVER,350.0,PCS,350.0,1.0,0.090,26.9,HELLO KITTY,NaN,NaN,NaN,NaN
3,PVC,4,BACK COVER,250.0,PCS,250.0,1.0,0.077,20.8,HELLO KITTY,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,1220.0,NaN,NaN,4.0,0.347,94.9,NaN,NaN,NaN,NaN,NaN


In [3]:
dataframe = dataframe.drop(columns=['BIS NO.', 'BIS MODEL NO.', 'MAH', 'MADE IN', 'LOGO'])
dataframe.dropna(inplace=True, subset=['MARK'])
dataframe.head()

,MARK,CTN NO,DESCRIPTION,PCS/TOTAL,UNITS,PCS/CTN,CTN/TOTAL,CBM/TOTAL,WEIGHT/TOTAL
0,PVC,1,BACK COVER,300.0,PCS,300.0,1.0,0.0900,22.8
1,PVC,2,BACK COVER,320.0,PCS,320.0,1.0,0.0900,24.4
2,PVC,3,BACK COVER,350.0,PCS,350.0,1.0,0.0900,26.9
3,PVC,4,BACK COVER,250.0,PCS,250.0,1.0,0.0770,20.8
6,VKIY,1,BACK COVER,698.0,PCS,698.0,1.0,0.1272,21.0


In [4]:
grouped = dataframe.groupby(['MARK', 'DESCRIPTION', 'PCS/CTN'])
print(grouped.head(10))

       MARK CTN NO DESCRIPTION  PCS/TOTAL UNITS  PCS/CTN  CTN/TOTAL  \
0       PVC      1  BACK COVER      300.0   PCS    300.0        1.0   
1       PVC      2  BACK COVER      320.0   PCS    320.0        1.0   
2       PVC      3  BACK COVER      350.0   PCS    350.0        1.0   
3       PVC      4  BACK COVER      250.0   PCS    250.0        1.0   
6      VKIY      1  BACK COVER      698.0   PCS    698.0        1.0   
..      ...    ...         ...        ...   ...      ...        ...   
316  RVC.EM      9  BACK COVER      500.0   PCS    500.0        1.0   
317  RVC.EM     10  BACK COVER      500.0   PCS    500.0        1.0   
407  RVC.EM    100  BACK COVER      600.0   PCS    600.0        1.0   
524  RVC.EM    217  BACK COVER      600.0   PCS    600.0        1.0   
600  RVC.EM    293  BACK COVER      400.0   PCS    400.0        1.0   

     CBM/TOTAL  WEIGHT/TOTAL  
0      0.09000         22.80  
1      0.09000         24.40  
2      0.09000         26.90  
3      0.07700         

In [5]:
dataframe['Block'] = (dataframe['PCS/CTN'] != dataframe['PCS/CTN'].shift()).cumsum()

# Step 2: Group by MARK, Block, and generate summaries
grouped = dataframe.groupby(['MARK', 'Block']).agg(
    First_CTN=('CTN NO', 'first'),
    Last_CTN=('CTN NO', 'last'),
    T_CTN=('CTN/TOTAL', 'sum'),
    WT=('WEIGHT/TOTAL', 'first'),
    UNIT=('UNITS', 'first'),
    QTY=('PCS/CTN', 'first'),
    T_QTY=('PCS/CTN', 'sum'),
    DESCRIPTION=('DESCRIPTION', 'first')
).reset_index()

# Step 3: Generate the CTN NO field
grouped['CTN NO'] = grouped.apply(
    lambda row: (
        f"{row['First_CTN'] if row['First_CTN'] is not None else 1} - "
        f"{row['Last_CTN'] if row['Last_CTN'] is not None else int(row['T_CTN'])}"
        if row['T_CTN'] > 1
        else f"{row['First_CTN'] if row['First_CTN'] is not None else 1}"
    ),
    axis=1
)
grouped['T.QTY'] = grouped['T_QTY']
grouped['T.CTN'] = grouped['T_CTN']
# Step 4: Reorder and clean up
final_columns = ['MARK', 'CTN NO', 'DESCRIPTION', 'T.CTN', 'QTY', 'UNIT', 'T.QTY', 'WT']
# grouped['DESCRIPTION'] = 'BACK COVER'  # Assuming the same description
final_dataset = grouped[final_columns]

# Display the final dataset
print(final_dataset)
final_dataset.to_excel('processed.xlsx', index=False)

            MARK   CTN NO    DESCRIPTION  T.CTN     QTY UNIT    T.QTY     WT
0         AARNAV    1 - 2     BACK COVER    2.0   199.0  PCS    199.0    NaN
1   AJ / SKM -55       55  UNLOCK MAGNET    1.0   100.0  PCS    100.0   6.70
2          DAKSH   1 - 15   LCD FRAME 中框   15.0   550.0  PCS   8250.0  15.55
3          DAKSH       16   LCD FRAME 中框    1.0   534.0  PCS    534.0  14.75
4          DAKSH  17 - 18   LCD FRAME 中框    2.0   700.0  PCS   1400.0  19.85
..           ...      ...            ...    ...     ...  ...      ...    ...
87          VKIY        3     BACK COVER    1.0   698.0  PCS    698.0  19.60
88          VKIY        4     BACK COVER    1.0   699.0  PCS    699.0  20.10
89         VRB50    1 - 4        PLOYBAG    4.0  5000.0  PCS  20000.0  20.00
90         VRB50    5 - 6     WATCH CELL    2.0  6000.0  PCS  12000.0  15.00
91         VRB50    7 - 8   PACKING  BOX    2.0   125.0  PCS    250.0   7.50

[92 rows x 8 columns]


In [ ]:
'''
Don't Run this Cell
'''

grouped_dataset = grouped.agg(
    First_CTN=('CTN NO', 'first'),  # First CTN NO
    Last_CTN=('CTN NO', 'last'),    # Last CTN NO
    T_CTN=('CTN/TOTAL', 'sum'),
    WT=('WEIGHT/TOTAL', 'first'),  # Weight/Total of the first element,
    UNIT=('UNITS', 'first')
).reset_index()

grouped_dataset.sort_values(by=['MARK', 'First_CTN'], inplace=True)

# Handle cases where Group_Count is 1
# grouped_dataset['CTN NO'] = grouped_dataset.apply(
#     lambda row: f"{row['First_CTN']}" if row['T_CTN'] == 1 else f"{row['First_CTN']} - {row['Last_CTN']}",
#     axis=1
# )
grouped_dataset['CTN NO'] = grouped_dataset.apply(
    lambda row: (
        f"{row['First_CTN'] if row['First_CTN'] is not None else 1} - "
        f"{row['Last_CTN'] if row['Last_CTN'] is not None else int(row['T_CTN'])}"
        if row['T_CTN'] > 1
        else f"{row['First_CTN'] if row['First_CTN'] is not None else 1}"
    ),
    axis=1
)
grouped_dataset['T.QTY'] = grouped_dataset['T_CTN'] * grouped_dataset['PCS/CTN']
grouped_dataset['T.CTN'] = grouped_dataset['T_CTN']
grouped_dataset['QTY'] = grouped_dataset['PCS/CTN']

# Drop intermediate columns to clean up
grouped_dataset.drop(columns=['First_CTN', 'Last_CTN', 'T_CTN', 'PCS/CTN'], inplace=True)

# Display the final dataset
desired_order = ['MARK', 'CTN NO', 'DESCRIPTION', 'T.CTN', 'QTY', 'UNIT', 'T.QTY', 'WT']
reordered_dataset = grouped_dataset[desired_order]
print(reordered_dataset)
reordered_dataset.to_excel('processed.xlsx', index=False)

            MARK             CTN NO    DESCRIPTION  T.CTN     QTY UNIT  \
0         AARNAV              1 - 2     BACK COVER    2.0   199.0  PCS   
1   AJ / SKM -55                 55  UNLOCK MAGNET    1.0   100.0  PCS   
6          DAKSH             1 - 15   LCD FRAME 中框   15.0   550.0  PCS   
5          DAKSH                 16   LCD FRAME 中框    1.0   534.0  PCS   
11         DAKSH            17 - 18   LCD FRAME 中框    2.0   700.0  PCS   
12         DAKSH                 19   LCD FRAME 中框    1.0   730.0  PCS   
10         DAKSH                 20   LCD FRAME 中框    1.0   650.0  PCS   
7          DAKSH                 21   LCD FRAME 中框    1.0   590.0  PCS   
8          DAKSH                 22   LCD FRAME 中框    1.0   600.0  PCS   
13         DAKSH                 23   LCD FRAME 中框    1.0   880.0  PCS   
3          DAKSH                 24   LCD FRAME 中框    1.0   470.0  PCS   
4          DAKSH                 25   LCD FRAME 中框    1.0   528.0  PCS   
9          DAKSH                 26   

In [6]:
consolidated = final_dataset.groupby(['DESCRIPTION'])
consolidated_dataset = consolidated.agg(
    Description=('DESCRIPTION', 'first'),
    Quantity=('T.QTY', 'sum'),
)
consolidated_dataset

,Description,Quantity
DESCRIPTION,,
BACK COVER,BACK COVER,188967.0
BATTERY,BATTERY,15600.0
LCD,LCD,17460.0
LCD FRAME 中框,LCD FRAME 中框,27731.0
MAT,MAT,700.0
PACKING BOX,PACKING BOX,250.0
PHONE STAND,PHONE STAND,240.0
PLOYBAG,PLOYBAG,20000.0
UNLOCK MAGNET,UNLOCK MAGNET,100.0


In [7]:
from openpyxl import load_workbook
from openpyxl.styles import Font, Alignment

# Load the existing Excel file
file_path = 'processed.xlsx'  # Replace with your file path
workbook = load_workbook(file_path)


# Load the first sheet (or specify the sheet name)
sheet_name = workbook.sheetnames[0]
sheet = workbook[sheet_name]
cambria_font = Font(name='Cambria', size=11)
for row in sheet.iter_rows():
    for cell in row:
        # Set the font
        cell.font = cambria_font
        # Remove borders by setting none (optional, usually default)
        cell.border = None
        # Align content centrally (optional, adjust as needed)
        cell.alignment = Alignment(horizontal="center", vertical="center")
# Define the starting cell
start_row = 6
start_col = 10  # Column 'J' corresponds to the 10th column
data = consolidated_dataset.values.tolist()
print(data)
headers = consolidated_dataset.columns.tolist()

# Write data
for row_idx, row in enumerate(data, start=start_row):
    for col_idx, value in enumerate(row, start=start_col):
        sheet.cell(row=row_idx, column=col_idx, value=value)
# Auto-adjust column widths
for column in sheet.columns:
    max_length = 0
    column_letter = column[0].column_letter  # Get column letter (e.g., A, B, C)
    for cell in column:
        if cell.value:  # Check if cell is not empty
            max_length = max(max_length, len(str(cell.value)))
    adjusted_width = max_length + 2  # Add extra space for padding
    sheet.column_dimensions[column_letter].width = adjusted_width
# Save the workbook
workbook.save(file_path)

print("Data appended successfully!")

[['BACK COVER', 188967.0], ['BATTERY', 15600.0], ['LCD', 17460.0], ['LCD FRAME 中框', 27731.0], ['MAT', 700.0], ['PACKING  BOX', 250.0], ['PHONE STAND', 240.0], ['PLOYBAG', 20000.0], ['UNLOCK MAGNET', 100.0], ['WATCH CELL', 12000.0]]
Data appended successfully!
